# PRACTICA-05: Análisis de Componentes Principales (PCA)

Principal Component Analysis (PCA) es un método estadístico que permite simplificar la complejidad de espacios muestrales con muchas dimensiones a la vez que conserva su información. Supóngase que existe una muestra con $n$ individuos cada uno con $p$ variables ($X_{1}, X_{2}, …, X_{p}$), es decir, el espacio muestral tiene $p$ dimensiones. PCA permite encontrar un número de factores subyacentes ($z<p$) que explican aproximadamente lo mismo que las $p$ variables originales. Donde antes se necesitaban $p$ valores para caracterizar a cada individuo, ahora bastan $z$ valores. Cada una de estas $z$  nuevas variables recibe el nombre de **componente principal**.

**Objetivo**: Reducir la dimensionalidad de un dataset y así simplificar el modelado posterior.

* Estandarizar los datos (para cada una de las $m$ observaciones)
* Obtener los vectores y valores propios a partir de la matriz de covarianzas o de correlaciones o incluso la técnica de Descomposición de Valores Singulares.
* Ordenar los valores propios en orden descendente y quedarnos con los $p$ que se correspondan a los $p$ mayores y así disminuir el número de variables del dataset ($p<m$)
* Constrir la matriz de proyección $W$ a partir de los $p$ vectores propios
* Transformar el dataset original $X$ a través de $W$ para así obtener datos en el subespacio dimensional de dimensión $p$, que será $Y$

## Dataset
El dataset de clasificación contiene las medidas de flores (en centímetros) que pertenecen a 3 especies diferentes:
* Iris-setosa
* Iris-versicolor
* Iris-virginica

Contiene 4 atributos o características. En particular, la **longitud** y la **anchura** de sus pétalos y sépalos expresadas en centímetros. Por tanto, se trata de un problema de 4 dimensiones.

In [ ]:
import IPython.display as display
from PIL import Image
display.display(Image.open('iris_petal-sepal.png'))

In [ ]:
# Carga de los datos y librerías
import pandas as pd

In [136]:
df = pd.read_csv("iris.csv")

In [ ]:
df.head()

### Gráfico de dispersión
El diagrama de dispersión es una buena manera de visualizar las correlaciones entre las características. Se examina la correlación de **Longitud del sépalo** con otras características. Entonces, **Longitud del sépalo** será nuestro eje $y$, otros se ubicarán en el eje $x$. Entonces se clasifican y se mantienen en marcos de datos distintos para ver las correlaciones claramente.

In [ ]:
data_sorted_bySW = df.sort_values('Sepal.Width')
data_sorted_byPL = df.sort_values('Petal.Length')
data_sorted_byPW = df.sort_values('Petal.Width')

In [ ]:
!pip install plotly
!pip install cufflinks

In [ ]:
import numpy as np
import pandas as pd
import plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

df2 = df.iloc[:100, :]

bySW = go.Scatter(
                    x = data_sorted_bySW['Sepal.Width'],
                    y = data_sorted_bySW['Sepal.Length'],
                    mode = "markers",
                    name = "Ancho del sépalo (cm)",
                    marker = dict(color = 'rgba(255, 0, 0, 0.9)'),
                    text = data_sorted_bySW.Species
)

byPL = go.Scatter(
                    x = data_sorted_byPL['Petal.Length'],
                    y = data_sorted_byPL['Sepal.Length'],
                    mode = "markers",
                    name = "Longitud del pétalo (cm)",
                    marker = dict(color = 'rgba(0, 255, 0, 0.9)'),
                    text = data_sorted_byPL.Species
)

byPW = go.Scatter(
                    x = data_sorted_byPW['Petal.Width'],
                    y = data_sorted_byPW['Sepal.Length'],
                    mode = "markers",
                    name = "Ancho del pétalo (cm)",
                    marker = dict(color = 'rgba(0, 0, 255, 0.9)'),
                    text = data_sorted_byPW.Species
)

layout = dict(title = 'Cambio de longitud del sépalo por otras propiedades',
              xaxis= dict(title= 'centimetros',ticklen= 5,zeroline= False)
             )
u = [bySW, byPL, byPW]
fig = dict(data = u, layout=layout)
iplot(fig)

* Parece que el **ancho del pétalo** y la **longitud del sépalo** tienen una correlación muy fuerte.
* Podemos decir que existe una correlación entre la **longitud del pétalo** y la **longitud del sépalo**, pero no como la anterior.
* No hay correlación entre la **longitud del sépalo** y el **ancho del sépalo**.

De manera que como existe una correlación lineal entre **ancho del pétalo** y **longitud del sépalo** una de las variables se podría excluir del proceso.

### Gráfica de dispersión 3D
Ahora se procede a analizar las tres variables restantes

In [ ]:
i_setosa = df[df['Species']  == 'setosa']
i_versicolor = df[df['Species']  == 'versicolor']
i_virginica = df[df['Species']  == 'virginica']
i_setosa.head()

In [ ]:
# Iris-setosa
trace_setosa = go.Scatter3d(
                        x = i_setosa['Sepal.Length'],
                        y = i_setosa['Sepal.Width'],
                        z = i_setosa['Petal.Length'],
                        mode = 'markers',
                        opacity = 0.7,
                        name = "setosa",
                        marker = dict(
                                    size = 5,
                                    color = 'rgba(255,102, 255,0.8)'
                        )
)

# Iris-versicolor
trace_versicolor = go.Scatter3d(
                        x = i_versicolor['Sepal.Length'],
                        y = i_versicolor['Sepal.Width'],
                        z = i_versicolor['Petal.Length'],
                        mode = 'markers',
                        opacity = 0.7,
                        name = "versicolor",
                        marker = dict(
                                    size = 5,
                                    color = 'rgba(102, 255, 51, 0.8)'
                        )
)

# Iris-virginica
trace_virginica = go.Scatter3d(
                        x = i_virginica['Sepal.Length'],
                        y = i_virginica['Sepal.Width'],
                        z = i_virginica['Petal.Length'],
                        mode = 'markers',
                        opacity = 0.7,
                        name = "virginica",
                        marker = dict(
                                    size = 5,
                                    color = 'rgba(51, 102, 255, 0.8)'
                        )
)

list_3d = [trace_setosa, trace_versicolor, trace_virginica]

fig_3d = go.Figure(data = list_3d)
iplot(fig_3d)

A continuación, dividimos el conjunto de datos en dos partes.

In [ ]:
#matriz de datos y categoría
X = df.iloc[:,0:4].values
# la submatriz "x" contiene los valores de las primeras 4 columnas del dataframe y todas las filas\n

y = df.iloc[:,4].values
# El vector "y" contiene los valores de la 4 columna (especie) para todas las filas

In [ ]:
X[0]

In [ ]:
y[0]

In [ ]:
import plotly.graph_objects as go
import plotly.tools as tls

In [ ]:
import matplotlib.pyplot as plt

i_setosa = df[df['Species']  == 'setosa']
i_versicolor = df[df['Species']  == 'versicolor']
i_virginica = df[df['Species']  == 'virginica']

plt.hist(i_setosa["Sepal.Length"], alpha=0.7, label='setosa',stacked=True)
plt.hist(i_versicolor["Sepal.Length"], alpha=0.7, label='versicolor',stacked=True)
plt.hist(i_virginica["Sepal.Length"], alpha=0.7, label='virginica',stacked=True)
plt.legend(loc='upper right')
plt.title("Longitud del sépalo")
plt.show()


plt.hist(i_setosa["Sepal.Width"], alpha=0.7, label='setosa')
plt.hist(i_versicolor["Sepal.Width"], alpha=0.7, label='versicolor')
plt.hist(i_virginica["Sepal.Width"], alpha=0.7, label='virginica')
plt.legend(loc='upper right')
plt.title("Ancho del sépalo")
plt.show()

plt.hist(i_setosa["Petal.Length"], alpha=0.7, label='setosa',stacked=True)
plt.hist(i_versicolor["Petal.Length"], alpha=0.7, label='versicolor')
plt.hist(i_virginica["Petal.Length"], alpha=0.7, label='virginica')
plt.legend(loc='upper right')
plt.title("Longitud del pétalo")
plt.show()


plt.hist(i_setosa["Petal.Width"], alpha=0.7, label='setosa')
plt.hist(i_versicolor["Petal.Width"], alpha=0.7, label='versicolor')
plt.hist(i_virginica["Petal.Width"], alpha=0.7, label='virginica')
plt.legend(loc='upper right')
plt.title("Ancho del pétalo")
plt.show()

# Normalización

Cuando las distintas características o atributos de un dataset están expresadas en distintas escalas se hace patente la necesidad de normalizar sus valores. En este caso, en el que las medidas de sépalos y pétalos están expresadas en centímetros, no sería imprescindible. Al aplicar esta técnica se asume que los datos de trabajo tienen una distribución gaussiana o normal. Por tanto, aplicamos a los datos una transformación de normalización de forma que su media sea igual a 0, y su varianza=1. 

In [ ]:
# Aplicamos una transformación de los datos para poder aplicar las propiedades de la distribución normal\n
from sklearn.preprocessing import StandardScaler

In [ ]:
# estandariza las variables (i.e., por columna) a media cero y varianza unitaria
X_std = StandardScaler().fit_transform(X)

In [ ]:
# estandariza las variables y grafica histogramas como los anteriores
dfX_std = pd.DataFrame(X_std)
dfX_std.head()

### 1- Calculamos la descomposición de valores y vectores propios
Los vectores propios son las direcciones en las que la varianza de los datos es mayor. La varianza  de una variable aleatoria es una medida de dispersión (definida como la esperanza del cuadrado de la desviación de dicha variable respecto a su media). Por tanto, las direcciones en las que la varianza es mayor, representan la esencia principal de la información contenida en el dataset, por eso se les llama Componentes Principales. Al igual que un vector propio es una dirección, el valor propio es un número, que representa el valor de la varianza sobre ese vector propio. Por ello, para encontrar las Componentes Principales que concentren esa esencia de la información del dataset, calcularemos primero la matriz de covarianza, que nos da la medida de dispersión conjunta entre variables.

##### a) Usando la Matriz de Covarianzas

In [ ]:
from IPython.display import display, Math, Latex

In [ ]:
display(Math(r'\sigma_{jk} = \frac{1}{n-1}\sum_{i=1}^m (x_{ij} - \overline{x_j})(x_{ik} - \overline{x_k})'))

In [ ]:
# Matriz de covarianzas
display(Math(r'\Sigma = \frac{1}{n-1}((X-\overline{x})^T(X-\overline{x}))'))

In [ ]:
# Vector promedio (vector de promedios)
display(Math(r'\overline{x} = \sum_{i=1}^n x_i\in \mathbb R^m'))

In [ ]:
import numpy as np

In [ ]:
# Observar que la media es casi cero
mean_vect = np.mean(X_std, axis=0)
mean_vect

In [ ]:
(X_std - mean_vect)[0]

In [ ]:
X_std[0]

In [ ]:
mean_vect

In [ ]:
cov_matrix = (X_std - mean_vect).T.dot((X_std - mean_vect))/(X_std.shape[0]-1)
print("La matriz de covarianza es \n%s"%cov_matrix)

In [ ]:
np.cov(X_std.T)

In [ ]:
eig_vals, eig_vectors = np.linalg.eig(cov_matrix)
print("Valores propios \n%s"%eig_vals)
print("Vectores propios \n%s"%eig_vectors)

##### b) Usando la Matriz de Correlaciones

In [ ]:
corr_matrix = np.corrcoef(X_std.T)
corr_matrix

In [ ]:
eig_vals_corr, eig_vectors_corr = np.linalg.eig(corr_matrix)
print("Valores propios \n%s"%eig_vals_corr)
print("Vectores propios \n%s"%eig_vectors_corr)

In [ ]:
corr_matrix = np.corrcoef(X.T)
corr_matrix

##### c) Descomposición de Valores Singulares

In [ ]:
u,s,v = np.linalg.svd(X_std.T)
u

In [ ]:
s

In [ ]:
v

### 2 - Las componentes principales

In [ ]:
# verifique que los vectores propios son unitarios (que forman una base al menos normal, 
# se podría pedir averiguar si es ortonormal)
for ev in eig_vectors:
    print("La longitud del VP es: %s"%np.linalg.norm(ev))

In [ ]:
eigen_pairs = [(np.abs(eig_vals[i]), eig_vectors[:,i]) for i in range(len(eig_vals))]
eigen_pairs

Ordenamos los vectores propios con valor propio de mayor a menor

In [ ]:
eigen_pairs.sort()
eigen_pairs.reverse()
eigen_pairs

Si lo que queremos es reducir la dimensionalidad del dataset, perdiendo la menor información posible, descartaremos los vectores propios cuyos valores propios sean más bajos, ya que son aquellos que menos información aportan al conjunto global.

In [ ]:
print("Valores propios en orden descendente:")
for ep in eigen_pairs:
    print(ep[0])

In [ ]:
total_sum = sum(eig_vals)
var_exp = [(i/total_sum)*100 for i in sorted(eig_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)

El objetivo de este caso es proyectar este dataset 4D en un espacio de menor dimensionalidad, para mejorar la eficiencia de cálculo, al mismo tiempo que se retiene la mayor parte de la información. La pregunta clave será ¿cuál va ser este valor? ¿3D?,   ¿2D?, ¿1D?. Para ello seguiremos el siguiente proceso.

Una vez ordenados los valores propios, que recordamos son una medida de la varianza de los datos, la cuestión es decidir, cuál es el menor número de vectores propios o componentes principales, con el que podemos expresar “la esencia principal” de la información contenida en ese dataset. Para ello, usaremos un métrica que se conoce como “varianza explicada”, que muestra cuánta varianza se puede atribuir a cada una de estas componentes principales.

In [ ]:
plot1 = go.Bar(x=["CP %s"%i for i in range(1,5)], y = var_exp, showlegend=False)
plot2 = go.Scatter(x=["CP %s"%i for i in range(1,5)], y = cum_var_exp, showlegend=True, name = "% de Varianza Explicada Acumulada")

data = go.Data([plot1, plot2])

layout = go.Layout(xaxis = go.XAxis(title="Componentes principales"), 
               yaxis = go.YAxis(title = "Porcentaje de varianza explicada"),
               title = "Porcentaje de variabilidad explicada por cada componente principal")

fig = go.Figure(data = data, layout = layout)
fig.show()

En la gráfica se aprecia claramente que la mayor parte de la varianza (en torno al 72%) corresponde a la primera componente. La segunda acumula algo más del 22% de la varianza, mientras que la tercera (3%) puede ser descartada sin perder demasiada información, ya que las dos primeras componentes explican más del 94% de la varianza.

En este ejemplo, construiremos una matriz de proyección que convertirá el conjunto de datos inicial (de 4D) en un conjunto de datos de 2D centrado en las sus componentes principales (las direcciones de los vectores propios correspondientes). De esta forma, la tarea de interpretar los patrones de información contenidos en los datos, será mucho más sencilla.

In [ ]:
W = np.hstack((eigen_pairs[0][1].reshape(4,1), 
               eigen_pairs[1][1].reshape(4,1)))
W

In [ ]:
X[0]

### 3- Proyectando las variables en el nuevo subespacio vectorial

In [ ]:
display(Math(r'Y = X \cdot W, X \in M(\mathbb R)_{150, 4}, W \in M(\mathbb R)_{4,2}, Y \in M(\mathbb R)_{150, 2}'))

In [ ]:
Y = X_std.dot(W)
Y

Y, por último, representamos gráficamente el nuevo espacio de datos, con éstas últimas líneas:

In [ ]:
results = []

for name in ('setosa', 'versicolor', 'virginica'):
    result = go.Scatter(x=Y[y==name,0], y = Y[y==name, 1], 
                    mode = "markers", name=name, 
                     marker=go.Marker(size = 12, line = go.Line(color='rgba(220,220,220,0.15)', width=0.5), opacity = 0.8))
    results.append(result)

data = go.Data(results)
layout = go.Layout(showlegend=True, scene =go.Scene(xaxis=go.XAxis(title="Componente Principal 1"),
                                             yaxis=go.YAxis(title="Componente Principal 2")))

fig = go.Figure(data=data, layout=layout)
fig.show()

Por lo tanto, se ha conseguido reducir el conjunto de datos de trabajo inicial a un conjunto de datos de dos dimensiones que aún así conserva la información más esencial, de forma que nos resultará mucho más sencillo el trabajo de crear, por ejemplo, un modelo de clasificación a partir de estos datos.

# EJERCICIOS

**(1) Trace un gráfico de barras que muestre el número de datos disponibles para cada especie en el conjunto de datos. Ponga un color diferente para cada especie**.

In [ ]:
df

In [ ]:
datos = df.groupby('Species').count().reset_index()
datos

In [ ]:
import plotly.express as px
fig = px.bar(datos, x='Species', y='Petal.Width',  color="Species")
fig.show()

**(2) Elabore un diagrama de caja (Box plot) del conjunto de datos, que nos muestre la representación visual de cómo se dispersan nuestros datos en el plano. Este método se utiliza en el análisis estadístico para comprender varias medidas, como la media, la mediana y la desviación**.

In [ ]:
versi = df[df['Species'] == 'versicolor']
setosa = df[df['Species'] == 'setosa']
virgi = df[df['Species'] == 'virginica']

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Box(
    y= versi['Sepal.Length'], 
    name = 'Sepal lenght'
))
fig.add_trace(go.Box(
    y= versi['Sepal.Width'], 
    name = 'Sepal widht'
))
fig.add_trace(go.Box(
    y= versi['Petal.Length'],
    name = 'Petal lenght'
))
fig.add_trace(go.Box(
    y= versi['Petal.Width'],
    name = 'Petal width'
))
fig.update_layout(title_text="Versicolor")


In [ ]:
fig = go.Figure()

fig.add_trace(go.Box(
    y= setosa['Sepal.Length'], 
    name = 'Sepal lenght'
))
fig.add_trace(go.Box(
    y= setosa['Sepal.Width'], 
    name = 'Sepal widht'
))
fig.add_trace(go.Box(
    y= setosa['Petal.Length'],
    name = 'Petal lenght'
))
fig.add_trace(go.Box(
    y= setosa['Petal.Width'],
    name = 'Petal width'
))
fig.update_layout(title_text="Setosa")

In [ ]:
fig = go.Figure()

fig.add_trace(go.Box(
    y= virgi['Sepal.Length'], 
    name = 'Sepal lenght'
))
fig.add_trace(go.Box(
    y= virgi['Sepal.Width'], 
    name = 'Sepal widht'
))
fig.add_trace(go.Box(
    y= virgi['Petal.Length'],
    name = 'Petal lenght'
))
fig.add_trace(go.Box(
    y= virgi['Petal.Width'],
    name = 'Petal width'
))
fig.update_layout(title_text="Veginica")

**(3) Muestre una gráfica que permita analizar la longitud del sépalo para cada especie. Esto es, analice cada atributo por separado para cada especie. Use un color diferente para cada especie**.

In [ ]:
# Vamos a elegir un bar box con la visualización de todos los puntos.
fig = px.box(df, x="Species", y="Sepal.Length",points="all", color = 'Species')
fig.show()

**(4) Trace una gráfica que indique el promedio de todas las longitudes de sépalos de especies de Iris. Use un color diferente para cada especie**.

In [146]:
df2 = df.copy()
df2['Species'] = 'setosa'
df_v = df2.copy()
df2['Species'] = 'versicolor'
df_v = pd.concat([df_v, df2])
df2['Species'] = 'virginica'
df_v = pd.concat([df_v, df2])

In [157]:
fig = go.Figure()

fig.add_trace(go.Violin(x=df['Species'],
                        y=df['Sepal.Length'],
                        scalegroup='Yes', name='Por Especie',
                        side='negative',
                        line_color='blue')
)
fig.add_trace(go.Violin(x=df_v['Species'],
                        y=df_v['Sepal.Length'],
                        scalegroup='No', name='Total',
                        side='positive',
                        line_color='orange')
)

fig.update_traces(meanline_visible=True)
fig.update_layout(violingap=0, violinmode='overlay')
fig.update_xaxes(fixedrange=True)
fig.update_xaxes(title_text='Especie')
fig.update_yaxes(title_text='Longitud')
fig.update_xaxes(range = [-0.5,2.5])
fig.show()

**(5) Elabore una gráfica que nos permita identificar cada especie con sus atributos. Esto es, una gráfica que me permita clasificar si una especie en particular es Iris-setosa, Iris-versicolor o Iris-virginica**.  

In [ ]:
# Esto se peude hacer por medio d eunna gráfica de correlaciones

fig = px.scatter_matrix(df,
    dimensions=["Sepal.Width", "Sepal.Length", "Petal.Width", "Petal.Length"],
    color="Species")
fig.show()

In [110]:
# Como vemos los mejores atributos que nos pueden dar una mejor clasificación son:
# petal widht, petal lenght y sepal widht. Así que grafiquemos en 3d

import plotly.graph_objects as go
import numpy as np

trace_setosa = go.Scatter3d(
                        x = i_setosa['Petal.Length'],
                        y = i_setosa['Petal.Width'],
                        z = i_setosa['Sepal.Width'],
                        mode = 'markers',
                        opacity = 0.7,
                        name = "setosa",
                        marker = dict(
                                    size = 5,
                                    color = 'rgba(255,102, 255,0.8)'
                        )
)

# Iris-versicolor
trace_versicolor = go.Scatter3d(
                        x = i_versicolor['Petal.Length'],
                        y = i_versicolor['Petal.Width'],
                        z = i_versicolor['Sepal.Width'],
                        mode = 'markers',
                        opacity = 0.7,
                        name = "versicolor",
                        marker = dict(
                                    size = 5,
                                    color = 'rgba(102, 255, 51, 0.8)'
                        )
)

# Iris-virginica
trace_virginica = go.Scatter3d(
                        x = i_virginica['Petal.Length'],
                        y = i_virginica['Petal.Width'],
                        z = i_virginica['Sepal.Width'],
                        mode = 'markers',
                        opacity = 0.7,
                        name = "virginica",
                        marker = dict(
                                    size = 5,
                                    color = 'rgba(51, 102, 255, 0.8)'
                        )
)

list_3d = [trace_setosa, trace_versicolor, trace_virginica]

fig_3d = go.Figure(data = list_3d)
fig_3d.update_layout(scene = dict(
    xaxis_title="Longitud de Pétalo",
    yaxis_title="Ancho de Pétalo",
    zaxis_title="Ancho de Sépalo")
    )

iplot(fig_3d)